In [1]:
import requests
import env
import os
import pandas as pd
import json
from env import Client_ID
from igdb.wrapper import IGDBWrapper


#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Connecting to API

In [ ]:
url = env.get_db_url()
response = requests.post(url)
print(response.text)


In [ ]:
data = response.json()
data.keys()

In [ ]:
access_token = data['access_token']

In [ ]:
# verify access token matches
access_token == data['access_token']

In [ ]:
requests.post(url)

In [ ]:
#function to connect to api and get access token
#requests.post(url) != 200
url = env.get_db_url()
response = requests.post(url)
data = response.json()
access_token = data['access_token']


In [3]:
def connect_api():
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    return access_token

### Making requests from API

- Most of the requests to the API will use the POST method
- The base URL is: https://api.igdb.com/v4
- You define which endpoint you wish to query by appending /{endpoint name} to the base URL eg. https://api.igdb.com/v4/games
- Include Client ID and access_token in the HEADER of your request so that your headers look like the following. Take special care of the capitalisation. Bearer should be hard-coded.

    ''' 
    Client-ID: Client ID

    Authorization: Bearer access_token 
    '''

- You use the BODY of your request to specify the fields you want to retrieve as well as any other filters, sorting etc

**Example**

    '''
    POST https://api.igdb.com/v4/games
    Client-ID: abcdefg12345
    Authorization: Bearer access12345token
    fields *;
    '''

more examples https://api-docs.igdb.com/#examples 

In [ ]:
url = env.get_db_url()
base_url = 'https://api.igdb.com/v4'
response = requests.post(base_url)
print(response.text)

In [ ]:
url = env.get_db_url()
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
base_url = 'https://api.igdb.com/v4'
response = requests.post(base_url , headers=headers)
print(response.text)

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; popularity;sort popularity desc;'
r = requests.post('https://api.igdb.com/v4/games' ,data=data ,  headers=headers)

In [ ]:
r.json()

In [ ]:
response.json()

In [ ]:
r.text

In [4]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; rating; sort rating asc;'
r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)

NameError: name 'access_token' is not defined

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )

In [9]:
wrapper = IGDBWrapper(f'{Client_ID}', f'{access_token}')

In [ ]:
'''With a wrapper instance already created'''
# JSON API request
byte_array = wrapper.api_request(
            'games',
            'fields *; limit 500;'
          )
# parse into JSON however you like...

# Protobuf API request
from igdb.igdbapi_pb2 import GameResult
byte_array = wrapper.api_request(
            'games.pb', # Note the '.pb' suffix at the endpoint
            'fields *; limit 500;'
          )
games_message = GameResult()
games_message.ParseFromString(byte_array) # Fills the protobuf message object with the response

In [ ]:
byte_array = wrapper.api_request(
            'games',
            'fields *; limit 500;'
          )

In [ ]:
testgames = []
for i in range(0, 500):
    games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 500};')
    testgames.append(games)

In [ ]:
testgames[0]

In [ ]:
pd.DataFrame(testgames.json())

In [ ]:
byte_array

In [ ]:
y = json.loads(byte_array)

In [ ]:
pd.DataFrame(y).head()

In [ ]:
pd.DataFrame(y).columns

In [ ]:
pd.DataFrame(y['id', 'category', 'genres', 'name', 'parent_game', 'slug','summary', 'tags', 'themes', 'url', 'websites',])

In [ ]:

pd.DataFrame(y).head(1)

In [ ]:
pd.DataFrame(y).url

In [ ]:
y

In [ ]:
games_message.ParseFromString(byte_array)

In [ ]:
type(games_message)

In [ ]:
games_message[1]

get ratings

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
#data = 'fields *; popularity;sort popularity desc;'
requests.get('https://api.igdb.com/v4/age_ratings' ,  headers=headers).text

### get games library

In [5]:
def connect_api():
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    return access_token

In [6]:
access_token = connect_api()

In [5]:
def connect_to_games():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; popularity;sort popularity desc;'
    r = requests.post('https://api.igdb.com/v4/games' ,data=data ,  headers=headers)
    return r

In [11]:
tf = connect_to_games()

In [6]:
def run_wrapper():
    wrapper = IGDBWrapper(f'{Client_ID}', f'{access_token}')
    return wrapper

In [8]:
wrapper = run_wrapper()

In [9]:
def get_game_library(wrapper):
    testgames2 = []
    for i in range(0, 10):
        games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 10};')
        testgames2.append(games)
    return testgames2

In [10]:
tf = get_game_library(wrapper)

In [57]:
# function that puts response list object into a dataframe for each page
def game_library(wrapper):
    game_library = pd.DataFrame()
    for i in range (0, 409):
        games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(games)
        results_df =pd.DataFrame(y)
        game_library = pd.concat([game_library, results_df])
    return game_library

In [58]:
game_library = game_library(wrapper)

In [59]:
game_library.head()

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,aggregated_rating,aggregated_rating_count,dlcs,multiplayer_modes,bundles,franchise,expansions,ports,expanded_games,remakes,remasters,standalone_expansions,forks
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...","[13196, 18623, 28244, 32510, 36258, 55036, 791...",panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,"[22, 268435471, 268435472]",[22],1654096076,https://www.igdb.com/games/panzer-corps-2-axis...,[68880],"[285491, 286755]",2992a43b-4d59-7ab8-e500-71a352404759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,0,NaN,1498435200,"[20127, 1988884]",[5],Dogou Souken,NaN,NaN,"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],NaN,1604620800,https://www.igdb.com/games/dogou-souken,NaN,NaN,5e7446ff-58d9-e744-f0d3-5e10241304c1,5.364576e+08,[52],[91007],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",[9],City Mysteries,NaN,"[155025, 155026, 155027]","[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],NaN,1641425677,https://www.igdb.com/games/city-mysteries,NaN,NaN,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,1.404346e+09,[37],[138333],[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]","[31, 32]",Sword of the Black Stone,NaN,"[231528, 231529, 231530, 231531, 231532]","[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",[1],1643857546,https://www.igdb.com/games/sword-of-the-black-...,NaN,[84617],0b575316-96dc-b056-86e0-c958c1c95efb,1.527120e+09,[6],[188385],NaN,"[155850, 155852]",[1],[2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,[33736],0,169077.0,1621959514,"[2052375, 2057300]","[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,"[463145, 463148, 463149, 463151, 463152]","[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",NaN,1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,NaN,[192858],9585e273-2606-970f-3088-60261ea77068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[63465],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
game_library.columns

Index(['id', 'artworks', 'category', 'cover', 'created_at', 'external_games', 'genres', 'name', 'parent_game', 'screenshots', 'similar_games', 'slug', 'summary', 'tags', 'themes', 'updated_at', 'url', 'videos', 'websites', 'checksum', 'first_release_date', 'platforms', 'release_dates', 'age_ratings', 'involved_companies', 'game_modes', 'player_perspectives', 'alternative_names', 'game_engines', 'keywords', 'status', 'rating', 'rating_count', 'storyline', 'total_rating', 'total_rating_count', 'version_parent', 'version_title', 'hypes', 'collection', 'franchises', 'follows', 'aggregated_rating', 'aggregated_rating_count', 'dlcs', 'multiplayer_modes', 'bundles', 'franchise', 'expansions', 'ports', 'expanded_games', 'remakes', 'remasters', 'standalone_expansions', 'forks'], dtype='object')

**Takeaways**

- delete  'artworks',  'cover', 'created_at', 'external_games', 'screenshots', 'similar_games',  'tags', 'updated_at', 'url', 'videos', 'websites', 'checksum',  'alternative_names',  'keywords', 'status', 'rating_count', 'storyline', 'total_rating_count',  'hypes', 'follows', 'aggregated_rating_count'

In [ ]:
game_library2.to_csv("games.csv")

### get genres

In [15]:
access_token = connect_api()

In [7]:
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/genres' ,data=data ,  headers=headers)

In [17]:
r

<Response [200]>

In [16]:
genres = wrapper.api_request('genres', 'fields *;limit 500;')

In [17]:
x = json.loads(genres)

In [18]:
genres =pd.DataFrame(x)

In [61]:
# function that puts response list object into a dataframe for each page
def get_genres(wrapper):
    genres = pd.DataFrame()
    for i in range (0, 409):
        genres = wrapper.api_request('genres', 'fields *; limit 500;')
        y = json.loads(games)
        results_df =pd.DataFrame(y)
        genres = pd.concat([genres, results_df])
    return genres


In [19]:
genres

,id,created_at,name,slug,updated_at,url,checksum
0,4,1297555200,Fighting,fighting,1323216000,https://www.igdb.com/genres/fighting,2ccc6572-bdde-6ed4-8843-25447ea40782
1,5,1297555200,Shooter,shooter,1323216000,https://www.igdb.com/genres/shooter,bb15fd3f-0f46-e5f3-2b40-d046cf9bd2ef
2,7,1297555200,Music,music,1323216000,https://www.igdb.com/genres/music,22d44a0d-89c7-580f-eef2-e06f178fdd47
3,8,1297555200,Platform,platform,1323216000,https://www.igdb.com/genres/platform,697fc5a4-b96f-a803-288a-498bd5dd1de1
4,9,1297555200,Puzzle,puzzle,1323216000,https://www.igdb.com/genres/puzzle,616de9c3-8a00-0232-9df9-00014cfac51b
5,10,1297555200,Racing,racing,1323216000,https://www.igdb.com/genres/racing,73c2653f-54d8-c1bd-d523-dd57fb753194
6,11,1297555200,Real Time Strategy (RTS),real-time-strategy-rts,1323216000,https://www.igdb.com/genres/real-time-strategy...,aaa36cbc-2258-8653-a461-1358df8ce445
7,12,1297555200,Role-playing (RPG),role-playing-rpg,1323216000,https://www.igdb.com/genres/role-playing-rpg,42dea3b2-7fe2-e734-91cd-f80ce62a14c3
8,13,1297555200,Simulator,simulator,1323216000,https://www.igdb.com/genres/simulator,9779772a-f08f-9e8e-bd26-70c9eecc34e8
9,14,1297555200,Sport,sport,1323216000,https://www.igdb.com/genres/sport,e598c458-3e21-7a14-50a5-ea53733ab22f


In [21]:
genres.columns

Index(['id', 'created_at', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

***Takeaways**

- drop  'created_at', 'updated_at', 'url', 'checksum'

- keep 'id', 'name', 'slug'

### get age ratings

In [30]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/age_ratings' ,data=data ,  headers=headers)

age_ratings = wrapper.api_request('age_ratings', 'fields *;limit 500;')

x = json.loads(age_ratings)

age_ratings =pd.DataFrame(x)

In [22]:
def get_age_ratings(wrapper):
    age_ratings = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('age_ratings', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        age_ratings = pd.concat([age_ratings, results_df])
    return age_ratings

In [23]:
age_ratings = get_age_ratings(wrapper)
age_ratings

,id,category,rating,checksum,content_descriptions,synopsis
0,22474,1,8,361fd414-94d8-d6f8-6641-47abd3e8a75c,NaN,NaN
1,43430,1,8,5970434e-a37f-d43d-8975-2b23ab3370b3,NaN,NaN
2,2853,1,10,2d21809b-c282-95ba-69fb-5eda4d684b43,"[4244, 4245, 4246, 4247]",NaN
3,32550,1,9,40557417-e2a9-9a4f-5810-4159463467a1,[32972],NaN
4,65037,3,15,9c0f7e9a-8ceb-27f3-3514-cc6407a176cd,"[66283, 66284, 66285]",NaN
...,...,...,...,...,...,...
261,92085,4,19,cf63d709-d55b-fb4c-7a70-1bcdaae55d7d,NaN,NaN
262,92087,2,2,68e0ea9a-3730-364d-e2ad-d067f8cf54d7,"[96108, 96109]",NaN
263,92088,3,13,fbe60226-d930-c21a-14f7-533e01b4abf7,NaN,NaN
264,92086,1,8,11bc1676-e4fa-181d-da66-3952076e5009,[96107],NaN


In [26]:
age_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69266 entries, 0 to 265
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    69266 non-null  int64 
 1   category              69266 non-null  int64 
 2   rating                69266 non-null  int64 
 3   checksum              69266 non-null  object
 4   content_descriptions  34662 non-null  object
 5   synopsis              7062 non-null   object
dtypes: int64(3), object(3)
memory usage: 3.7+ MB


In [24]:
age_ratings.columns

Index(['id', 'category', 'rating', 'checksum', 'content_descriptions', 'synopsis'], dtype='object')

**TakeAways**

- delete  'checksum', 'content_descriptions', 'synopsis'
- keep id', 'category', 'rating'

### get age rating contents

In [54]:
def get_age_rating_desc(wrapper):
    age_ratings_desc = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('age_rating_content_descriptions', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        age_ratings_desc = pd.concat([age_ratings_desc, results_df])
    return age_ratings_desc

In [55]:
age_rating_desc = get_age_rating_desc(wrapper)
age_rating_desc

,id,category,description,checksum
0,47,21.0,Strong Language,befd2777-5035-29b9-a592-3e0ba572b3bb
1,20,21.0,Strong Language,a0f5a6b5-3f93-00bc-f733-c5480af42132
2,50,21.0,Strong Language,21d63306-18a3-73e9-1a02-922431d31b0f
3,35,3.0,Blood,18821468-e01a-5629-67f8-36539a6cc69a
4,39,21.0,Strong Language,24807756-487a-29aa-8858-7781e3ce5e06
...,...,...,...,...
44,96106,71.0,Language,aba99669-8751-f00d-d848-983605d1374a
45,96105,69.0,Violence,950c5977-052d-1889-eb85-ddd363f82fa5
46,96109,53.0,Fear,b9c50df8-8811-e57f-19f6-0845f8476e47
47,96107,6.0,Comic Mischief,9c26bbec-c1e1-9914-0357-3b837df7f52a


In [56]:
age_rating_desc.columns

Index(['id', 'category', 'description', 'checksum'], dtype='object')

**Takeaways**

- delete checksum 

### get characters

In [76]:
access_token = connect_api()

In [27]:
def get_characters(wrapper):
    characters = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('characters', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        characters = pd.concat([characters, results_df])
    return characters

In [28]:
characters = get_characters(wrapper)
characters

,id,created_at,games,name,slug,updated_at,url,checksum,gender,mug_shot,species,description,akas
0,13116,1555977600,[103020],Nuclear Launch Silo,nuclear-launch-silo,1555977600,https://www.igdb.com/characters/nuclear-launch...,bb6801a9-2b8f-6843-1964-2f440f1b8e48,NaN,NaN,NaN,NaN,NaN
1,14447,1562198400,[75242],Agent Carson,agent-carson,1562198400,https://www.igdb.com/characters/agent-carson,ac16307c-58d1-37cd-cda7-490f9345bb7b,NaN,NaN,NaN,NaN,NaN
2,504,1397865600,"[2765, 2911]",Gary,gary,1501632000,https://www.igdb.com/characters/gary,56dfd9a7-5cfa-ee91-14e5-46a493843313,NaN,NaN,NaN,NaN,NaN
3,2692,1414800000,[41],O,o,1458518400,https://www.igdb.com/characters/o,e32722d2-9798-5a72-b240-fdcbe43fabc5,NaN,NaN,NaN,NaN,NaN
4,6715,1449878400,[1958],Thophile Paddington,thophile-paddington,1449878400,https://www.igdb.com/characters/thophile-paddi...,51e57c03-5b7e-fc22-c838-1f7dfe753e92,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,16575,1647897710,[2369],Menelag,menelag,1647897729,https://www.igdb.com/characters/menelag,ecc219c8-c15f-7ee2-4bb5-8287ff53b418,NaN,NaN,NaN,NaN,NaN
392,16579,1647897711,[2369],Additional,additional,1647897732,https://www.igdb.com/characters/additional,075e65d8-c756-a0c9-9224-086ccfd9cfe3,NaN,NaN,NaN,NaN,NaN
393,4629,1432043872,[897],Cate Archer,cate-archer,1651237111,https://www.igdb.com/characters/cate-archer,1dbdc5b9-346d-5870-20c7-3627a132292e,1.0,4068.0,1.0,NaN,"[Catherine Ann Archer, The Fox]"
394,10236,1475914520,NaN,Funtime Freddy,funtime-freddy,1657731347,https://www.igdb.com/characters/funtime-freddy,3b31ca09-177a-0c37-eb02-06f979aa5f96,0.0,4069.0,4.0,Funtime Freddy is a major antagonist who first...,NaN


In [30]:
characters.columns

Index(['id', 'created_at', 'games', 'name', 'slug', 'updated_at', 'url', 'checksum', 'gender', 'mug_shot', 'species', 'description', 'akas'], dtype='object')

**Takeaways**

- delete  'created_at',  'updated_at', 'url', 'checksum', 'gender', 'mug_shot', 'species', 'description', 'akas'

- keep 'id', 'games', 'name', 'slug'

### get collections

In [32]:
def get_collections(wrapper):
    collections = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('collections', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        collections = pd.concat([collections, results_df])
    return collections

In [33]:
collections = get_collections(wrapper)
collections

,id,created_at,games,name,slug,updated_at,url,checksum
0,8202,1661995850,[189115],Grey Heritage,grey-heritage,1662330684,https://www.igdb.com/collections/grey-heritage,2e258dc4-d204-b8a8-df3a-29451234bc6b
1,7852,1651653419,[109972],Elsword,elsword,1651653420,https://www.igdb.com/collections/elsword,fe3864e8-5e0c-f27c-07cd-5ce3e4b3ffcf
2,4117,1507934083,"[11933, 18423, 39908, 61311, 61312, 61314]",Sprint,sprint,1507934083,https://www.igdb.com/collections/sprint,d0e1c01e-5c30-6ddb-2969-033fb82966f5
3,3247,1507276944,"[59261, 59263, 59264]",Hitozuma Unyu Saimin NTR Ben,hitozuma-unyu-saimin-ntr-ben,1507276944,https://www.igdb.com/collections/hitozuma-unyu...,fca09763-036f-9276-a361-2bb1b02365eb
4,5297,1516127813,"[77413, 77414, 77415, 77416, 77417]",LavaCat,lavacat,1516127813,https://www.igdb.com/collections/lavacat,34ee825d-4905-8420-5e0b-6400fe5c4826
...,...,...,...,...,...,...,...,...
302,206,1321478702,"[843, 981, 1963, 1964, 1966, 1967, 1968, 2925,...",Rayman,rayman,1663458606,https://www.igdb.com/collections/rayman,cb49c23c-9a09-bd9d-ef27-301679c1cb5d
303,717,1391348655,"[4598, 4622, 6351, 6494, 9545, 11300, 11522, 2...",Contra,contra,1663458759,https://www.igdb.com/collections/contra,9f8ba156-09f7-b651-dd68-f1ff0b1dd8e7
304,3061,1506587780,"[67197, 67198, 68312, 68313, 68314, 68332, 683...",Arcade Archives,arcade-archives,1663317638,https://www.igdb.com/collections/arcade-archives,0304b232-0d3c-bbd3-680d-dc0cbe058589
305,8281,1663322592,[207041],Hitsuji Shitsuji,hitsuji-shitsuji,1663329271,https://www.igdb.com/collections/hitsuji-shitsuji,136fdf3b-577d-bb95-963f-ac9346ba92a2


In [34]:
collections.columns

Index(['id', 'created_at', 'games', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**
- delete 'created_at', 'updated_at', 'url', 'checksum'
- keep 'id', 'games', 'name', 'slug'

### get franchises

In [39]:
def get_franchises(wrapper):
    franchises = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('franchises', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        franchises = pd.concat([franchises, results_df])
    return franchises

In [42]:
franchises = get_franchises(wrapper)
franchises

,id,created_at,games,name,slug,updated_at,url,checksum
0,545,1428781176,"[6805, 9626, 10091, 11754, 11755, 11767, 11768...",VOCALOID,vocaloid,1428781176,https://www.igdb.com/franchises/vocaloid,d7a62e0a-1662-f2e4-1dde-1fc10c829e39
1,2574,1617402728,[145390],NextRev,nextrev,1617402728,https://www.igdb.com/franchises/nextrev,ac73431c-640d-8b37-c757-09c2135e1167
2,2798,1623025704,[151890],Tuned Turtle,tuned-turtle,1623025704,https://www.igdb.com/franchises/tuned-turtle,3220ee67-be4b-ce96-dd2e-241cfa976809
3,783,1458155907,[18385],Austin Powers,austin-powers,1458155907,https://www.igdb.com/franchises/austin-powers,c2e2e449-0b19-7bc3-9ed2-ceba6a5844aa
4,730,1446474129,NaN,Kane & Lynch,kane-and-lynch,1446474129,https://www.igdb.com/franchises/kane-and-lynch,81914463-391d-da66-9f96-3cd8fb085c2c
...,...,...,...,...,...,...,...,...
187,4000,1662152560528,NaN,Wangan Midnight,wangan-midnight,1662152560,https://www.igdb.com/franchises/wangan-midnight,71f53437-0788-51ea-8266-fd0fbda7acd6
188,4006,1662655852703,NaN,Joey and Penguin,joey-and-penguin,1662655852,https://www.igdb.com/franchises/joey-and-penguin,cc39cbfe-697b-6209-3008-113ac97833be
189,51,1372173591416,"[138, 189, 190, 390, 422, 542, 2050, 2157, 217...",LEGO,lego,1663230468,https://www.igdb.com/franchises/lego,73dbe2df-dc2e-2be9-d6cd-144dff577638
190,4032,1663324223545,NaN,LEX Studios,lex-studios,1663324223,https://www.igdb.com/franchises/lex-studios,7bac82ac-f258-f502-cc8f-1e5fd77a7079


In [41]:
franchises = pd.DataFrame()

In [44]:
franchises.columns

Index(['id', 'created_at', 'games', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

- delete  'created_at', 'updated_at', 'url', 'checksum'
- keep 'id', 'games', 'name', 'slug'

### get game engines

In [49]:
def get_game_engines(wrapper):
    game_engines = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_engines', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_engines = pd.concat([game_engines, results_df])
    return game_engines

In [48]:
game_engines = pd.DataFrame()

In [50]:
game_engines = get_game_engines(wrapper)
game_engines

,id,companies,created_at,logo,name,platforms,slug,updated_at,url,checksum
0,186,[5025],1425427200,31.0,OEngine,"[6, 48, 49]",oengine,1425427200,https://www.igdb.com/game_engines/oengine,7b4118c7-5081-d1de-8fa7-c122074254da
1,1410,[14081],1649979323,NaN,AI5,"[13, 118, 121, 149]",ai5,1652857462,https://www.igdb.com/game_engines/ai5,6bb11504-0a8e-67e1-ff46-46c70fc177bd
2,1291,NaN,1627144462,NaN,Pinball Construction Set,NaN,pinball-construction-set,1627506462,https://www.igdb.com/game_engines/pinball-cons...,2b15d014-4f7e-fcb0-addf-6c5c9c653388
3,237,[356],1431907200,NaN,The Madness Engine,NaN,the-madness-engine,1578700800,https://www.igdb.com/game_engines/the-madness-...,bf7f5406-4568-4aaf-b2b5-7c4bc106e3a3
4,975,NaN,1590935612,NaN,TIC-80,NaN,tic-80,1643881103,https://www.igdb.com/game_engines/tic-80,5b658b0b-b4a1-82e4-78fc-189ab707eede
...,...,...,...,...,...,...,...,...,...,...
129,820,NaN,1577779095,NaN,LithTech Jupiter EX,NaN,lithtech-jupiter-ex,1650853805,https://www.igdb.com/game_engines/lithtech-jup...,b7ff44d5-3226-0e2f-11ca-3c49935aa113
130,79,[6932],1411945309,0.0,GameMaker: Studio,"[3, 6, 9, 14, 34, 39, 46, 48, 49, 74]",gamemaker-studio,1663572164,https://www.igdb.com/game_engines/gamemaker-st...,58d6cd85-b8be-f581-67c9-72f2abcee2fd
131,626,NaN,1553127899,NaN,LithTech Jupiter,NaN,lithtech-jupiter,1650856108,https://www.igdb.com/game_engines/lithtech-jup...,6c6d9db1-43c3-0d79-e481-56317a7ae281
132,296,[7466],1452509450,43.0,Clausewitz Engine,NaN,clausewitz-engine,1651025009,https://www.igdb.com/game_engines/clausewitz-e...,b6e16986-c822-60e8-b70d-f26b3031764a


In [52]:
game_engines.columns

Index(['id', 'companies', 'created_at', 'logo', 'name', 'platforms', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

- delete  'created_at', 'logo',  'updated_at', 'url', 'checksum'

### endpoints

In [ ]:
endpoints = [
    'https://api.igdb.com/v4/age_ratings',
    'https://api.igdb.com/v4/age_rating_content_descriptions',
    'https://api.igdb.com/v4/characters',
    'https://api.igdb.com/v4/companies',
    'https://api.igdb.com/v4/collections',
    'https://api.igdb.com/v4/franchises',
    'https://api.igdb.com/v4/game_engines'
    ]

### get game modes

In [78]:
def get_game_modes(wrapper):
    game_modes = pd.DataFrame()
    for i in range (0, 1):
        a_ratings = wrapper.api_request('game_modes', 'fields *; limit 500;')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_modes = pd.concat([game_modes, results_df])
    return game_modes

In [79]:
game_modes = get_game_modes(wrapper)
game_modes

,id,created_at,name,slug,updated_at,url,checksum
0,1,1298937600,Single player,single-player,1323216000,https://www.igdb.com/game_modes/single-player,a43b1688-f968-3541-0897-9735ffde1745
1,2,1298937600,Multiplayer,multiplayer,1323216000,https://www.igdb.com/game_modes/multiplayer,288b548c-11e4-d910-f037-21d4e6a33b38
2,3,1298937600,Co-operative,co-operative,1323216000,https://www.igdb.com/game_modes/co-operative,e8329d55-33e6-f7ae-ae09-2848cb7ccc90
3,4,1298937600,Split screen,split-screen,1323216000,https://www.igdb.com/game_modes/split-screen,98247b75-6f46-7c77-521f-0945c684d842
4,5,1298937600,Massively Multiplayer Online (MMO),massively-multiplayer-online-mmo,1323216000,https://www.igdb.com/game_modes/massively-mult...,307d1126-6e3b-9215-06f1-10c8ecce05b4
5,6,1588809600,Battle Royale,battle-royale,1588809600,https://www.igdb.com/game_modes/battle-royale,bfc388ac-0502-ba05-af25-3dcc94a0acf3


In [81]:
game_modes.columns

Index(['id', 'created_at', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

- delete  'created_at', 'updated_at', 'url', 'checksum'

### get game version

In [88]:
def get_game_versions(wrapper):
    game_versions = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_versions', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_versions = pd.concat([game_versions, results_df])
    return game_versions

In [84]:
game_versions = get_game_versions(wrapper)
game_versions

,id,created_at,game,updated_at,url,checksum,features,games
0,241,1596585600,26192,1596585600,https://www.igdb.com/game_versions/the-last-of...,e71699ef-97ce-af28-c6b1-a5272393fcfa,NaN,NaN
1,8,1505174400,31311,1505174400,https://www.igdb.com/game_versions/monsti/,4db869ec-e1a3-3f1d-8d88-04d5609d21db,NaN,NaN
2,7,1505174400,20668,1505174400,https://www.igdb.com/game_versions/zombie-driver/,f2b2058d-7c6a-a793-5a88-bfc63871005b,NaN,NaN
3,13,1505174400,52983,1505174400,https://www.igdb.com/game_versions/eternal-jou...,9f694e8f-b4f7-2e97-479a-5190b328bd0d,NaN,NaN
4,10,1505174400,32721,1505174400,https://www.igdb.com/game_versions/zion/,55f1ff2a-7605-4bfc-da2b-ac7781971b5f,NaN,NaN
...,...,...,...,...,...,...,...,...
186,206,1573171200,13664,1573171200,https://www.igdb.com/game_versions/distraint/,6a1f626a-879d-d1bd-2390-3b6e4b2f9f0b,NaN,NaN
187,44,1505952000,7349,1505952000,https://www.igdb.com/game_versions/mass-effect...,d749f321-1e08-5674-438a-13df9e354ada,"[99, 100, 101, 102, 103]","[7349, 27766, 27767]"
188,45,1506038400,27421,1535500800,https://www.igdb.com/game_versions/middle-eart...,eca61d61-f746-ad65-915f-5e0fe7637738,"[253, 254, 255, 256, 257, 258, 259, 260, 261, ...","[27421, 36986, 53358, 53359, 106764]"
189,1,1505174400,28540,1507420800,https://www.igdb.com/game_versions/assassins-c...,4ff85042-d876-4aff-5589-25167ce699a7,"[50, 51, 52, 98, 135, 136, 137, 138, 139, 140,...","[28540, 39034, 39044, 39045, 39046, 39047]"


In [85]:
game_versions.columns

Index(['id', 'created_at', 'game', 'updated_at', 'url', 'checksum', 'features', 'games'], dtype='object')

**Takeawys**

- delete  'created_at', 'updated_at', 'url', 'checksum', 

### get game version features

In [89]:
def get_game_version_features(wrapper):
    game_version_features = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_version_features', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_version_features = pd.concat([game_version_features, results_df])
    return game_version_features

In [90]:
game_version_features = get_game_version_features(wrapper)
game_version_features

,id,category,description,position,title,values,checksum
0,337,0.0,Sharpens your aim with rifles and pistols. Alw...,1,DLC: The Sharpshooter Detective Suit,"[1385, 1386, 1387, 1388]",a10d3ebf-ae3a-f0cc-fc68-116b1545a9ac
1,347,0.0,NaN,0,"25 cm (10"") figurine of an Alb Mage","[1419, 1420]",2a42989b-4936-75bc-6331-0ce79abb225a
2,352,0.0,NaN,5,A making of DVD,"[1429, 1430]",bb46d55f-7145-0b5c-2f69-cea9dd2570d6
3,375,0.0,Double sided print of the in-game island,2,Collectible map,"[1519, 1520, 1521, 1522, 1523]",8a9ea6d0-b5f1-ece2-cd5d-61636d639ad0
4,382,0.0,"Exclusive to TESCO in the UK, it included thre...",9,Combat Strike Pack,"[1554, 1555, 1556, 1557, 1558]",263031ce-acb2-69a2-ab66-240dbaad4f4d
...,...,...,...,...,...,...,...
440,17,NaN,NaN,0,Season Pass,NaN,c27cf85d-4af5-9fc1-6f59-52a9624f9490
441,456,0.0,created in collaboration with Dark Horse Comic...,1,Full-color print comic book,"[1773, 1774, 1775, 1776]",218da9c1-cc23-cc6d-a50a-72ba78203f66
442,459,0.0,Full-color printed map of the world of Conan E...,4,World Map,"[1785, 1786, 1787, 1788]",3f282aa8-abeb-8fb8-2dc7-5277c451c0aa
443,555,0.0,NaN,0,Limited edition figurine,"[2154, 2155]",a655f38f-4322-fb48-6d4e-8bc71f3834c0


In [91]:
game_version_features.columns

Index(['id', 'category', 'description', 'position', 'title', 'values', 'checksum'], dtype='object')

**Takeaways**

-  delete  'checksum'

### get version feature values

In [ ]:
def get_version_features_values():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; limit 500;'
    r = requests.post('https://api.igdb.com/v4/game_version_feature_values' ,data=data ,  headers=headers)
    game_version_feature_values = wrapper.api_request('game_version_feature_values', 'fields *;limit 500;')
    x = json.loads(game_version_feature_values)
    game_versions_feature_values =pd.DataFrame(x)
    return game_versions_feature_values

In [92]:
def get_game_version_feature_values(wrapper):
    game_version_feature_values = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_version_feature_values', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_version_feature_values = pd.concat([game_version_feature_values, results_df])
    return game_version_feature_values

In [93]:
game_version_feature_values = get_game_version_feature_values(wrapper)
game_version_feature_values

,id,game,game_feature,included_feature,checksum,note
0,1581,52617,391,0.0,1db62035-9135-3d01-25a5-9c194d8fbd8f,NaN
1,1590,52617,394,0.0,e2e097d5-c88d-d6ea-48cf-1f5288f6bbe2,NaN
2,591,39045,144,0.0,6ee07864-0f1b-d41f-e2a5-e7dc2731d9a0,NaN
3,1987,83886,503,2.0,612d4e1b-a3b2-60e4-680a-064c0837f71e,NaN
4,1600,19333,399,0.0,ba6dcc29-45c7-6412-ebce-da6731910fb9,NaN
...,...,...,...,...,...,...
17,1983,53012,503,2.0,74eacd56-d0fa-62ab-a961-df386c4cd140,NaN
18,2155,193391,555,1.0,2eb547da-7c57-147e-1bdd-9d9b8dcc3eac,NaN
19,2154,5116,555,0.0,d257f647-5899-a6e0-fca6-516809965ba1,NaN
20,2156,5116,556,0.0,bc29ab0a-9945-f712-2de9-752bb23c3524,NaN


In [97]:
game_version_feature_values.columns

Index(['id', 'game', 'game_feature', 'included_feature', 'checksum', 'note'], dtype='object')

Takeaways

- delete 'checksum' 'note'

### get companies

In [98]:
def get_companies(wrapper):
    companies= pd.DataFrame()
    for i in range (0, 409):
        company = wrapper.api_request('companies', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(company)
        results_df =pd.DataFrame(y)
        companies = pd.concat([companies, results_df])
    return companies

In [99]:
companies = get_companies(wrapper)
companies

,id,change_date_category,country,created_at,description,developed,logo,name,slug,start_date,start_date_category,updated_at,url,websites,checksum,published,change_date,changed_company_id,parent
0,13710,7,752.0,1508976000,Tomolo Games develope fun and simple mobile ga...,"[74974, 74975, 74977]",1396.0,Tomolo Games,tomolo-games,1.494634e+09,0,1508976000,https://www.igdb.com/companies/tomolo-games,[1089],656407de-863f-13bc-82c0-16079a13c964,NaN,NaN,NaN,NaN
1,15094,7,233.0,1520899200,NaN,[26472],NaN,ZA/UM,za-slash-um,NaN,7,1573948800,https://www.igdb.com/companies/za-slash-um,"[1944, 3431]",d32b01ce-2272-3bf2-e809-1f16cf5c86d9,[26472],NaN,NaN,NaN
2,17841,7,246.0,1548720000,NaN,[114715],NaN,Moido Games,moido-games,NaN,7,1548720000,https://www.igdb.com/companies/moido-games,[1683],e48ef63d-fdc0-35a2-bbc6-db61277e2c25,NaN,NaN,NaN,NaN
3,16305,7,208.0,1535673600,"PortaPlay is an independent developer, residin...",[107175],1748.0,PortaPlay,portaplay,1.181952e+09,0,1536019200,https://www.igdb.com/companies/portaplay,[1323],5d28752a-62f8-f617-f0d5-6ac59c0efa6e,NaN,NaN,NaN,NaN
4,4270,7,392.0,1418428800,T's Music (ティーズ ミュージック Tiizu Myuujikku) is a g...,NaN,3304.0,T's Music,ts-music,-6.213568e+10,2,1576627200,https://www.igdb.com/companies/ts-music,[3517],238c892a-7f60-78f2-415a-7f114ccda898,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,43205,7,NaN,1663132230,Wheat is a Aspiring game dev a TF2Classic deve...,[217612],6693.0,Wheat,wheat,NaN,7,1663137702,https://www.igdb.com/companies/wheat,[6266],ddba68cf-1bcc-db60-1ce7-a23858656ca7,NaN,NaN,NaN,NaN
496,43207,7,578.0,1663153197,Hello Bard A/S is an independent game developm...,[37019],NaN,Hello Bard,hello-bard,1.483142e+09,2,1663154687,https://www.igdb.com/companies/hello-bard,[6267],58cb7354-fa27-a025-ed0d-0e7dae37e356,[37019],NaN,NaN,NaN
497,34124,7,250.0,1621274945,NaN,NaN,6696.0,Nobilis Interactive,nobilis-interactive,1.199059e+09,2,1663154776,https://www.igdb.com/companies/nobilis-interac...,NaN,8a1256de-6ebd-5240-bbba-bb5782a29930,[148055],NaN,NaN,NaN
498,43209,7,40.0,1663171356,Graphic design & Animation Company,NaN,6698.0,JoonyTv,joonytv,1.647994e+09,0,1663179583,https://www.igdb.com/companies/joonytv,[6269],1a53e5a1-6488-09df-36a2-83decc22bb8a,NaN,NaN,NaN,NaN


In [100]:
companies.columns

Index(['id', 'change_date_category', 'country', 'created_at', 'description', 'developed', 'logo', 'name', 'slug', 'start_date', 'start_date_category', 'updated_at', 'url', 'websites', 'checksum', 'published', 'change_date', 'changed_company_id', 'parent'], dtype='object')

**Takeaways**

- delete 'id', 'change_date_category', 'country', 'created_at', 'description', 'developed', 'logo', 'name', 'slug', 'start_date', 'start_date_category', 'updated_at', 'url', 'websites', 'checksum', 'published', 'change_date', 'changed_company_id', 'parent'

### get involved companies

In [159]:
def get_involved_companies(wrapper):
    involved_companies= pd.DataFrame()
    for i in range (0, 409):
        i_company = wrapper.api_request('involved_companies', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(i_company)
        results_df =pd.DataFrame(y)
        involved_companies = pd.concat([involved_companies, results_df])
    return involved_companies

In [161]:
involved_companies = get_involved_companies(wrapper)
involved_companies

,id,company,created_at,developer,game,porting,publisher,supporting,updated_at,checksum
0,95000,14486,1585267200,True,131992,False,False,False,1585440000,43e91be1-592f-c3be-3e1d-447b3f17584c
1,78799,1250,1557532800,True,20689,False,False,False,1557532800,169b9e17-f2e5-ba93-4b58-97ec89978c68
2,20543,19,1419033600,True,602,False,True,False,1419033600,3058ba74-b6d7-f755-d603-1ff10337c15f
3,36564,10142,1466121600,True,19671,False,False,False,1466294400,d88519f6-e9bb-fd6b-40a9-c58ccad78312
4,66470,2888,1531440000,True,105048,False,False,False,1531440000,77bc32cf-a18b-ef18-8099-ed3edb7e4ca2
...,...,...,...,...,...,...,...,...,...,...
246,178623,112,1657711901,False,45730,False,True,False,1657714382,5f448d4d-9be8-2bb0-0f2c-d93343c70919
247,178635,1274,1657714406,False,49977,False,True,False,1657714419,52e2b7d6-3a1f-9854-1499-d759591add28
248,178639,1274,1657715287,False,91775,False,True,False,1657715331,99ff7e26-b656-8cb2-15fd-87fe4595c690
249,185668,2064,1662084802,False,215085,False,True,False,1662085372,2079637d-afe6-22be-9c2c-d34961c93f80


In [162]:
involved_companies.columns

Index(['id', 'company', 'created_at', 'developer', 'game', 'porting', 'publisher', 'supporting', 'updated_at', 'checksum'], dtype='object')

**Takeaways**

### get multi player modes

In [163]:
def get_multi_player_modes(wrapper):
    multi_player_modes= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('multiplayer_modes', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        multi_player_modes = pd.concat([multi_player_modes, results_df])
    return multi_player_modes

In [164]:
multi_player_modes = get_multi_player_modes(wrapper)
multi_player_modes

,id,campaigncoop,dropin,game,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum,offlinecoopmax,onlinecoopmax,onlinemax
0,9953,False,False,92273,False,False,2.0,False,22.0,False,1b94d937-dc0c-c62d-10af-e58d42860bd8,NaN,NaN,NaN
1,1832,True,True,7153,False,True,0.0,False,12.0,False,2b09f64f-b8b9-6948-ad98-9ab9bede8e3b,2.0,0.0,0.0
2,7987,False,False,57887,False,True,2.0,False,49.0,False,56c9ee4c-aae3-ba3c-ead0-9e899e1c5267,0.0,0.0,0.0
3,7,False,False,46076,False,False,30.0,False,6.0,False,a41bcf9c-976d-8690-629f-4c2737f65bf1,NaN,NaN,NaN
4,10207,False,False,31256,False,False,NaN,False,82.0,False,788009fd-9e08-5826-577b-9844575ba2d9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,20758,False,False,218148,False,False,4.0,False,6.0,False,61dd0156-6010-37ff-f3b2-447f826d0251,NaN,NaN,NaN
237,20759,False,False,218148,False,False,4.0,False,14.0,False,3e55b3c6-d994-e73d-6ae9-80551b13d429,NaN,NaN,NaN
238,20768,False,False,218177,True,False,NaN,False,3.0,False,d189dd86-7196-0f1c-e8bf-b4b171d9e381,NaN,NaN,NaN
239,20769,False,False,218177,True,False,NaN,False,6.0,False,f5cc1041-df1c-ba2e-d359-941950f003cf,NaN,NaN,NaN


In [165]:
multi_player_modes.columns

Index(['id', 'campaigncoop', 'dropin', 'game', 'lancoop', 'offlinecoop', 'offlinemax', 'onlinecoop', 'platform', 'splitscreen', 'checksum', 'offlinecoopmax', 'onlinecoopmax', 'onlinemax'], dtype='object')

**Takeaways**

### get families

In [166]:
def get_platform_families(wrapper):
    platform_families= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platform_families', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platform_families = pd.concat([platform_families, results_df])
    return platform_families

In [167]:
platform_families = get_platform_families(wrapper)
platform_families

,id,name,slug,checksum
0,5,Nintendo,nintendo,398e5c87-f7db-6265-6da6-433441385208
1,4,Linux,linux,be8b89fa-823c-790a-1d5a-7e74315c1b27
2,2,Xbox,xbox,5bb9bcec-4aa7-45b8-129b-a6f88932942e
3,3,Sega,sega,bd1a9cc4-3632-33b0-ff4b-feab8cd15001
4,1,PlayStation,playstation,eadc0208-7972-9364-88b9-d2a9de75a998


In [168]:
platform_families.columns

Index(['id', 'name', 'slug', 'checksum'], dtype='object')

**Takeaways**

### get platform v release dates

In [172]:
def get_platform_version_releases(wrapper):
    platform_version_release_dates= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platform_version_release_dates', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platform_version_release_dates = pd.concat([platform_version_release_dates, results_df])
    return platform_version_release_dates

In [174]:
platform_version_release_dates = get_platform_version_releases(wrapper)
platform_version_release_dates

,id,category,date,human,m,region,y,checksum,updated_at
0,555,2,567907200,1987,12,8,1987,186c07ec-0d93-dc0f-6c4a-5fdb5a6138fa,NaN
1,338,0,1189555200,"Sep 12, 2007",9,3,2007,15e5b0aa-5bb6-ef30-ee85-0ef94cb3581f,NaN
2,334,0,1125532800,"Sep 01, 2005",9,7,2005,a4c921d5-1894-9a40-d20f-05959cd69a38,NaN
3,331,0,1415923200,"Nov 14, 2014",11,3,2014,3ebb8dc8-b8fa-f11a-b729-228b9dac35c4,NaN
4,333,0,1389830400,"Jan 16, 2014",1,7,2014,a15ecea5-cb71-916f-5bec-66ad7f571b19,NaN
...,...,...,...,...,...,...,...,...,...
391,660,0,1547942400,"Jan 20, 2019",1,7,2019,29210c4c-26c6-212c-e0ec-c0d6bdd23be9,1.660316e+09
392,659,0,1413763200,"Oct 20, 2014",10,2,2014,ddac84c3-8f5a-8372-fee3-5e05cc9517f5,1.660316e+09
393,661,2,725760000,1992,12,8,1992,d6cb796b-0679-d99b-30d8-decdf573187d,1.660320e+09
394,662,0,945043200,"Dec 13, 1999",12,5,1999,6fa54af0-85f8-088f-294c-d95dbbbb2a70,1.660322e+09


In [175]:
platform_version_release_dates.columns

Index(['id', 'category', 'date', 'human', 'm', 'region', 'y', 'checksum', 'updated_at'], dtype='object')

**Takeaways**

### get platform version companies

In [177]:
def get_platform_version_companies(wrapper):
    platform_version_release_companies= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platform_version_companies', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platform_version_release_companies = pd.concat([platform_version_release_companies, results_df])
    return platform_version_release_companies

In [180]:
platform_version_release_companies = get_platform_version_companies(wrapper)

platform_version_release_companies


,id,company,developer,manufacturer,checksum,comment
0,286,11986,True,True,e5035f31-713a-1036-2968-dd353b74db3b,NaN
1,283,10922,False,True,6cd3e57c-2757-cc07-94a6-35ff7085e4c9,NaN
2,282,14429,False,True,172b6f24-a49c-c304-6fb3-78f6bf120a19,NaN
3,281,12385,True,False,d6051460-e7dc-4e77-80ca-e5df378cd304,NaN
4,280,19831,True,True,2f509e1b-75d2-36db-4129-4bf89d50343b,NaN
...,...,...,...,...,...,...
424,650,40695,True,True,34204aec-5719-bc70-eaa9-80d649d4acac,NaN
425,651,70,True,False,c5b2ca2d-92e3-0d15-c40c-66603c6773fe,NaN
426,652,42680,True,True,aaf4cd13-c03f-83ae-4e84-2d9d98fad050,NaN
427,653,42681,True,False,4b080df5-0b43-df63-7d2d-938a3dcbd8a9,NaN


In [181]:
platform_version_release_companies.columns

Index(['id', 'company', 'developer', 'manufacturer', 'checksum', 'comment'], dtype='object')

**Takeaways**

### get platforms

In [179]:
def get_platforms(wrapper):
    platforms= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platforms', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platforms = pd.concat([platforms, results_df])
    return platforms

In [182]:
platforms = get_platforms(wrapper)
platforms

,id,alternative_name,category,created_at,name,platform_logo,slug,updated_at,url,versions,websites,checksum,generation,platform_family,abbreviation,summary
0,158,Commodore Dynamic Total Vision,6.0,1510012800,Commodore CDTV,292.0,commodore-cdtv,1522972800,https://www.igdb.com/platforms/commodore-cdtv,[223],[26],5d48648e-283c-e2df-df7d-6d3cbc7e0a58,NaN,NaN,NaN,NaN
1,339,Kids Computer Pico,1.0,1595808000,Sega Pico,NaN,sega-pico,1595808000,https://www.igdb.com/platforms/sega-pico,[456],NaN,6e867a5b-a071-fc7b-945b-a0622f64a4bc,4.0,3.0,NaN,NaN
2,8,PS2,1.0,1297555200,PlayStation 2,254.0,ps2,1414972800,https://www.igdb.com/platforms/ps2,"[58, 114]",NaN,be3e025c-d4ed-b795-443c-a8cb55fbfe17,6.0,1.0,PS2,NaN
3,39,NaN,4.0,1317686400,iOS,248.0,ios,1391644800,https://www.igdb.com/platforms/ios,[43],NaN,fefe4a2b-7c90-0e89-e811-902ea1cf2b58,NaN,NaN,iOS,NaN
4,94,NaN,6.0,1414195200,Commodore Plus/4,310.0,c-plus-4,1468454400,https://www.igdb.com/platforms/c-plus-4,[108],NaN,ea138789-6bbe-bcca-dff5-7e20e762f65a,NaN,NaN,C+4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,408,WG-108,5.0,1660155539,Mega Duck/Cougar Boy,687.0,mega-duck-slash-cougar-boy,1660326569,https://www.igdb.com/platforms/mega-duck-slash...,[548],[307],4bd967b3-9dff-84ab-809a-fdf191e5e0f5,4.0,NaN,NaN,NaN
191,410,Jag CD,1.0,1660245561,Atari Jaguar CD,688.0,atari-jaguar-cd,1660326828,https://www.igdb.com/platforms/atari-jaguar-cd,[550],[310],704539d3-9f54-2d0f-220c-986c484c76b0,5.0,NaN,NaN,NaN
192,415,NaN,5.0,1660319701,Watara/QuickShot Supervision,691.0,watara-slash-quickshot-supervision,1660326748,https://www.igdb.com/platforms/watara-slash-qu...,[555],[314],4ed0c0ee-3103-4d53-c0dc-612a391de28d,4.0,NaN,NaN,NaN
193,414,LeapTV VCD,1.0,1660314942,LeapTV,690.0,leaptv,1660326997,https://www.igdb.com/platforms/leaptv,[554],[313],74a35c87-865a-52ee-29f0-70c03039ff84,8.0,NaN,NaN,NaN


In [183]:
platforms.columns

Index(['id', 'alternative_name', 'category', 'created_at', 'name', 'platform_logo', 'slug', 'updated_at', 'url', 'versions', 'websites', 'checksum', 'generation', 'platform_family', 'abbreviation', 'summary'], dtype='object')

**Takeaways**

### get release dates

In [184]:
def get_release_dates(wrapper):
    release_dates= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('release_dates', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        release_dates = pd.concat([release_dates, results_df])
    return release_dates

In [185]:
release_dates = get_release_dates(wrapper)
release_dates

,id,category,created_at,date,game,human,m,platform,region,updated_at,y,checksum
0,170111,0,1558826090,1.552954e+09,76870,"Mar 19, 2019",3.0,48,8,1558826519,2019.0,eda4c3fc-5ce2-5c43-9a96-bfd20b24ab88
1,165447,0,1550867412,1.561421e+09,115477,"Jun 25, 2019",6.0,6,8,1550867489,2019.0,3f7a3186-1622-4da6-fb05-d223cea14454
2,85558,0,1497948084,5.660928e+08,38035,"Dec 10, 1987",12.0,121,5,1497948122,1987.0,df4b2c61-916b-d823-cdf7-286f06aeac62
3,26259,0,1422372749,1.079654e+09,2473,"Mar 19, 2004",3.0,24,1,1422377563,2004.0,b71a705c-751f-f832-18e5-1485482906db
4,310515,0,1631498978,6.317568e+08,48629,"Jan 08, 1990",1.0,99,5,1631499382,1990.0,7b655a08-c93d-d413-6ba7-8f5946acf34c
...,...,...,...,...,...,...,...,...,...,...,...,...
495,331150,7,1639701154,NaN,109582,TBD,NaN,6,8,1639701339,NaN,ff8bd362-af93-2abf-6120-7f06f62fc186
496,331153,7,1639701154,NaN,109582,TBD,NaN,14,8,1639701341,NaN,ca753dca-2830-d1c4-307d-98dc89b8e7f4
497,331155,0,1639701214,1.613606e+09,61467,"Feb 18, 2021",2.0,48,8,1639701338,2021.0,90ef6b66-8efa-b169-b905-eba7972c2285
498,331148,2,1639701093,1.640909e+09,28881,2021,12.0,161,8,1639701343,2021.0,3311a13c-476a-d350-2999-7d90462ed734


In [186]:
release_dates.columns

Index(['id', 'category', 'created_at', 'date', 'game', 'human', 'm', 'platform', 'region', 'updated_at', 'y', 'checksum'], dtype='object')

**Takeaways**

### get platform versions

In [187]:
def get_platform_versions(wrapper):
    platform_versions= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platform_versions', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platform_versions = pd.concat([platform_versions, results_df])
    return platform_versions

In [189]:
platform_versions = get_platform_versions(wrapper)
platform_versions

,id,companies,cpu,graphics,memory,name,os,platform_logo,platform_version_release_dates,slug,sound,storage,summary,url,checksum,media,resolutions,online,output,connectivity,main_manufacturer
0,103,[533],MOS Technology 6502 @ 1 MHz,monochrome character graphics,4–96 KB,Initial version,Commodore BASIC 1.0 ~ 4.0,543.0,[552],initial-version,none or beeper,"cassette tape, 5.25-inch floppy, 8-inch floppy...",The Commodore PET is a line of home/personal c...,https://www.igdb.com/platforms/cpet/version/in...,df0a5020-309d-04c9-5368-b2b9244bb6b4,NaN,NaN,NaN,NaN,NaN,NaN
1,19,NaN,Motorola 68000 @ 7.16 MHz (NTSC) 7.09 MHz (PAL),NaN,512 kB (9 MB maximum),Amiga A 500,AmigaOS v1.2 - 1.3,391.0,[134],amiga-a-500,4× 8-bit channels PCM at max 28 kHz with 6-bit...,NaN,"In 1982, the Amiga Corporation developed ""Lorr...",https://www.igdb.com/platforms/amiga/version/a...,8be75330-7847-49d4-7812-cae1b58d8e63,880 kB floppy disks,"736x567i 4-bpp PAL (736x483i 4 bpp NTSC), 368x...",NaN,NaN,NaN,NaN
2,282,[527],NaN,NaN,NaN,Switch Lite,NaN,534.0,[359],switch-lite,NaN,NaN,"A smaller version of the Switch, which only su...",https://www.igdb.com/platforms/switch/version/...,24097bf5-9bbf-5959-931b-4b87ece75fb8,NaN,NaN,NaN,NaN,NaN,NaN
3,139,[144],Ricoh 5A22 @ 3.58 MHz,NaN,128 KB,Super Famicom (SHVC-001),NaN,105.0,[204],super-famicom-shvc-001,"Nintendo S-SMP 8 channels, stereo, 64 KB",NaN,Original Japanese version (SHVC-001),https://www.igdb.com/platforms/snes--1/version...,3fcc6ae9-ac67-dc57-a433-780e25ee615f,ROM cartridge,256x224 to 512x448,"Satellaview (satellite radio), XBAND,","composite video, S-Video and RGB",NaN,NaN
4,164,NaN,NaN,NaN,NaN,Initial version,NaN,372.0,NaN,initial-version,NaN,NaN,NaN,https://www.igdb.com/platforms/trs-80/version/...,93cf119c-a0d5-5de9-fcaa-4692e0bd605c,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,553,[647],393 MHz ARM9-based processor,NaN,64 MB DDR SDRAM,Initial version,Linux,NaN,[658],initial-version,NaN,512 MB,The Leapster Explorer is a handheld console de...,https://www.igdb.com/platforms/leapster-explor...,ddd842c0-db98-ec71-1bae-ef855816426e,ROM Cartridge,320x240,NaN,NaN,NaN,NaN
308,554,"[648, 649]",1 GHz quad core custom processor,NaN,1GB DDR3 RAM,Initial version,NaN,690.0,"[659, 660]",initial-version,NaN,16 GB,LeapTV is an educational video game console de...,https://www.igdb.com/platforms/leaptv/version/...,0d18d1da-477d-77af-8331-f4c17ffd1afb,"ROM Cartridge, Video CD (Malaysia only)","1280x720, 640x480 (Malaysia only)",LeapFrog,"HDMI, composite video (Malaysia only)","Wi-Fi, Bluetooth",NaN
309,555,[650],4 MHz 8-bit 65SC02,NaN,NaN,Initial version,NaN,691.0,[661],initial-version,"2 Tonal + 1 Noise Channel, additional audio DM...",NaN,"The Watara Supervision, also known as the Quic...",https://www.igdb.com/platforms/watara-slash-qu...,52336677-f6d7-2067-f113-8cc6586b23bf,ROM Cartridge,160x160 in 4 color greyscale,NaN,TV Adpater,Two Player Link using DE-9 connector,NaN
310,556,"[651, 652]",NaN,NaN,NaN,Initial version,NaN,692.0,[662],initial-version,NaN,Rewritable magnetic disk,The Nintendo 64DD is a magnetic floppy disk dr...,https://www.igdb.com/platforms/nintendo-64dd/v...,401ff879-d203-c235-1c1a-ee8d5cac8c40,64 MB magnetic disk,NaN,Randnet,NaN,22.8 kbps dial-up modem,NaN


In [190]:
platform_versions.columns

Index(['id', 'companies', 'cpu', 'graphics', 'memory', 'name', 'os', 'platform_logo', 'platform_version_release_dates', 'slug', 'sound', 'storage', 'summary', 'url', 'checksum', 'media', 'resolutions', 'online', 'output', 'connectivity', 'main_manufacturer'], dtype='object')

**Takeaways**

### get player perspectives

In [188]:
def get_player_perspectives(wrapper):
    player_perspectives= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('player_perspectives', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        player_perspectives = pd.concat([player_perspectives, results_df])
    return player_perspectives

In [191]:
player_perspectives = get_player_perspectives(wrapper)
player_perspectives

,id,created_at,name,slug,updated_at,url,checksum
0,1,1298937600,First person,first-person,1323216000,https://www.igdb.com/player_perspectives/first...,4e23cb22-7a70-effb-b8e1-151317c6cdbd
1,2,1298937600,Third person,third-person,1323216000,https://www.igdb.com/player_perspectives/third...,2788b856-580c-66d0-bef3-d6169034f175
2,3,1298937600,Bird view / Isometric,bird-view-slash-isometric,1582502400,https://www.igdb.com/player_perspectives/bird-...,83c59132-6edd-150f-a25f-a86ddf6a0da3
3,5,1321228800,Text,text,1323216000,https://www.igdb.com/player_perspectives/text,e2df3906-bca8-e0d1-ca3b-67e1c5cfb334
4,4,1298937600,Side view,side-view,1323216000,https://www.igdb.com/player_perspectives/side-...,2d038ac6-36c5-af5b-83fd-11a88a638631
5,7,1462233600,Virtual Reality,virtual-reality,1462233600,https://www.igdb.com/player_perspectives/virtu...,963f8a54-d40f-1c9d-3bf4-d15267043693
6,6,1413158400,Auditory,auditory,1582502400,https://www.igdb.com/player_perspectives/auditory,74528cf1-64c3-cd78-18df-fae1594ee664


In [192]:
player_perspectives.columns

Index(['id', 'created_at', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

### get themes

In [194]:
def get_themes(wrapper):
    themes= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('themes', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        themes = pd.concat([themes, results_df])
    return themes

In [195]:
themes = get_themes(wrapper)
themes

,id,created_at,name,slug,updated_at,url,checksum
0,20,1322524800,Thriller,thriller,1323216000,https://www.igdb.com/themes/thriller,4ebeaa43-4165-5d23-142a-9bb2a26c7024
1,18,1322524800,Science fiction,science-fiction,1323216000,https://www.igdb.com/themes/science-fiction,fffad274-fee4-e195-eb78-72b2a35d643e
2,1,1322524800,Action,action,1323216000,https://www.igdb.com/themes/action,c177e0ff-f29a-2a2e-fce5-f945258ceb59
3,19,1322524800,Horror,horror,1323216000,https://www.igdb.com/themes/horror,28762ccf-724f-56cf-3784-dbf16dfcf174
4,21,1322524800,Survival,survival,1323216000,https://www.igdb.com/themes/survival,01d5251e-746d-d9b6-6a47-99e8e9fda8ff
5,17,1322524800,Fantasy,fantasy,1323216000,https://www.igdb.com/themes/fantasy,454bdc25-3dbf-5648-1e11-fc67ffc893aa
6,22,1322524800,Historical,historical,1323216000,https://www.igdb.com/themes/historical,1256e1bd-db05-276a-16ad-9704f7a56dcc
7,23,1322524800,Stealth,stealth,1323216000,https://www.igdb.com/themes/stealth,6c0cc20c-b7fc-0007-8ae0-4cc91301e9f1
8,27,1322524800,Comedy,comedy,1323216000,https://www.igdb.com/themes/comedy,2d128835-a3c1-7979-d561-005e651ada8e
9,28,1322524800,Business,business,1323216000,https://www.igdb.com/themes/business,b088a483-b1e9-685b-dbfb-4e37b58ef69d


In [196]:
themes.columns

Index(['id', 'created_at', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

## Prepare

### list initeration of genres

In [101]:
genres

,id,created_at,name,slug,updated_at,url,checksum
0,4,1297555200,Fighting,fighting,1323216000,https://www.igdb.com/genres/fighting,2ccc6572-bdde-6ed4-8843-25447ea40782
1,5,1297555200,Shooter,shooter,1323216000,https://www.igdb.com/genres/shooter,bb15fd3f-0f46-e5f3-2b40-d046cf9bd2ef
2,7,1297555200,Music,music,1323216000,https://www.igdb.com/genres/music,22d44a0d-89c7-580f-eef2-e06f178fdd47
3,8,1297555200,Platform,platform,1323216000,https://www.igdb.com/genres/platform,697fc5a4-b96f-a803-288a-498bd5dd1de1
4,9,1297555200,Puzzle,puzzle,1323216000,https://www.igdb.com/genres/puzzle,616de9c3-8a00-0232-9df9-00014cfac51b
5,10,1297555200,Racing,racing,1323216000,https://www.igdb.com/genres/racing,73c2653f-54d8-c1bd-d523-dd57fb753194
6,11,1297555200,Real Time Strategy (RTS),real-time-strategy-rts,1323216000,https://www.igdb.com/genres/real-time-strategy...,aaa36cbc-2258-8653-a461-1358df8ce445
7,12,1297555200,Role-playing (RPG),role-playing-rpg,1323216000,https://www.igdb.com/genres/role-playing-rpg,42dea3b2-7fe2-e734-91cd-f80ce62a14c3
8,13,1297555200,Simulator,simulator,1323216000,https://www.igdb.com/genres/simulator,9779772a-f08f-9e8e-bd26-70c9eecc34e8
9,14,1297555200,Sport,sport,1323216000,https://www.igdb.com/genres/sport,e598c458-3e21-7a14-50a5-ea53733ab22f


In [125]:
genreslist = genres[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)

In [112]:
game_library.head(1)

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,aggregated_rating,aggregated_rating_count,dlcs,multiplayer_modes,bundles,franchise,expansions,ports,expanded_games,remakes,remasters,standalone_expansions,forks
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...","[13196, 18623, 28244, 32510, 36258, 55036, 791...",panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,"[22, 268435471, 268435472]",[22],1654096076,https://www.igdb.com/games/panzer-corps-2-axis...,[68880],"[285491, 286755]",2992a43b-4d59-7ab8-e500-71a352404759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
test_library = game_library[['id' , 'genres']].head(5)

In [136]:
genres_dict = genreslist.set_index('id').to_dict()['slug']
genres_dict

{2: 'point-and-click',
 4: 'fighting',
 5: 'shooter',
 7: 'music',
 8: 'platform',
 9: 'puzzle',
 10: 'racing',
 11: 'real-time-strategy-rts',
 12: 'role-playing-rpg',
 13: 'simulator',
 14: 'sport',
 15: 'strategy',
 16: 'turn-based-strategy-tbs',
 24: 'tactical',
 25: 'hack-and-slash-beat-em-up',
 26: 'quiz-trivia',
 30: 'pinball',
 31: 'adventure',
 32: 'indie',
 33: 'arcade',
 34: 'visual-novel',
 35: 'card-and-board-game',
 36: 'moba'}

In [145]:
def test_funct(random_list):
    new_list = [genres_dict[i] for i in random_list]
    return new_list

In [147]:
test_funct([13, 15, 31, 32])

['simulator', 'strategy', 'adventure', 'indie']

In [140]:
[genres_dict[i] for i in [15,16]]

['strategy', 'turn-based-strategy-tbs']

In [153]:
test2 = pd.DataFrame(test_library.genres.apply(test_funct))
test2

,genres
0,"[strategy, turn-based-strategy-tbs]"
1,[shooter]
2,[puzzle]
3,"[adventure, indie]"
4,"[simulator, strategy, adventure, indie]"


In [154]:
test_library

,id,genres
0,202489,"[15, 16]"
1,40104,[5]
2,85031,[9]
3,99234,"[31, 32]"
4,148930,"[13, 15, 31, 32]"


In [156]:
test_library['genres'] = pd.DataFrame(test_library.genres.apply(test_funct))
test_library

,id,genres,nom
0,202489,"[strategy, turn-based-strategy-tbs]","[strategy, turn-based-strategy-tbs]"
1,40104,[shooter],[shooter]
2,85031,[puzzle],[puzzle]
3,99234,"[adventure, indie]","[adventure, indie]"
4,148930,"[simulator, strategy, adventure, indie]","[simulator, strategy, adventure, indie]"


In [157]:
game_library['genres'] = pd.DataFrame(game_library.genres.apply(test_funct))

TypeError: 'float' object is not iterable

In [158]:
game_library.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204500 entries, 0 to 499
Data columns (total 55 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       204500 non-null  int64  
 1   artworks                 67847 non-null   object 
 2   category                 204500 non-null  int64  
 3   cover                    145493 non-null  float64
 4   created_at               204500 non-null  int64  
 5   external_games           194624 non-null  object 
 6   genres                   158986 non-null  object 
 7   name                     204500 non-null  object 
 8   parent_game              20038 non-null   float64
 9   screenshots              138254 non-null  object 
 10  similar_games            165343 non-null  object 
 11  slug                     204500 non-null  object 
 12  summary                  169734 non-null  object 
 13  tags                     165367 non-null  object 
 14  themes 